In [4]:
import sys
import os
import numpy as np
import zipfile
from tqdm import tqdm
import scrapbook as sb
from tempfile import TemporaryDirectory
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.models.deeprec.deeprec_utils import download_deeprec_resources
from recommenders.models.newsrec.newsrec_utils import prepare_hparams
from recommenders.models.newsrec.models.lstur import LSTURModel
from recommenders.models.newsrec.io.mind_iterator import MINDIterator
from recommenders.models.newsrec.newsrec_utils import get_mind_data_set

print("System version:{}".format(sys.version))
print("Tensorflow version:{}".format(tf.__version__))

System version:3.8.17 (default, Jul  5 2023, 20:44:21) [MSC v.1916 64 bit (AMD64)]
Tensorflow version:2.6.1


In [24]:
import numpy as np

In [19]:
!pip install keras==2.6

In [12]:
!pip install protobuf==3.20.* --user

In [20]:
!pip install scrapbook

In [5]:
!pip install tensorflow==2.6.1

  Using cached numpy-1.19.5-cp38-cp38-win_amd64.whl (13.3 MB)
  Using cached typing_extensions-3.7.4.3-py3-none-any.whl (22 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.7.1
    Uninstalling typing_extensions-4.7.1:
      Successfully uninstalled typing_extensions-4.7.1
  Attempting uninstall: numpy
    Found existing installation: numpy 1.20.2
    Uninstalling numpy-1.20.2:
      Successfully uninstalled numpy-1.20.2


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\super\\AppData\\Local\\Temp\\pip-uninstall-l1dm46c2\\libopenblas.gk7gx5keq4f6uyo3p26ulgbqyhgqo7j4.gfortran-win_amd64.dll'
Consider using the `--user` option or check the permissions.



In [26]:
!pip install tensorflow

  Using cached numpy-1.19.5-cp38-cp38-win_amd64.whl (13.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.20.3
    Uninstalling numpy-1.20.3:
      Successfully uninstalled numpy-1.20.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
matplotlib 3.7.2 requires numpy>=1.20, but you have numpy 1.19.5 which is incompatible.
numba 0.57.1 requires numpy<1.25,>=1.21, but you have numpy 1.19.5 which is incompatible.
pandas 1.5.3 requires numpy>=1.20.3; python_version < "3.10", but you have numpy 1.19.5 which is incompatible.


In [28]:
!pip install numpy==1.20.3 --user

  Using cached numpy-1.20.3-cp38-cp38-win_amd64.whl (13.7 MB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.57.1 requires numpy<1.25,>=1.21, but you have numpy 1.20.3 which is incompatible.
tensorflow 2.6.1 requires numpy~=1.19.2, but you have numpy 1.20.3 which is incompatible.


In [27]:
conda uninstall numpy

Solving environment: ...working... failed

Note: you may need to restart the kernel to use updated packages.


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/msys2/noarch/repodata.json HTTP/1.1" 304 0
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/r/noarch/repodata.json HTTP/1.1" 304 0
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/main/win-64/repodata.json HTTP/1.1" 304 0
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/r/win-64/repodata.json HTTP/1.1" 304 0
DEBUG:urllib3.connectionpool:https://repo

In [1]:
!pip show numpy

Name: numpy
Version: 1.20.3
Summary: NumPy is the fundamental package for array computing with Python.
Home-page: https://www.numpy.org
Author: Travis E. Oliphant et al.
Author-email: 
License: BSD
Location: c:\users\super\appdata\roaming\python\python38\site-packages
Requires: 
Required-by: Bottleneck, category-encoders, contourpy, cornac, h5py, Keras-Preprocessing, lightfm, lightgbm, matplotlib, numba, opt-einsum, pandas, pandera, patsy, powerlaw, pyarrow, recommenders, scikit-learn, scikit-surprise, scipy, seaborn, statsmodels, tensorboard, tensorflow, transformers


In [23]:
epochs=5
seed=40
batch_size=64

MIND_type='demo'

In [24]:

data_path = r"C:\Users\super\OneDrive\Documents\SSIS\Swoopo"

train_news_file = os.path.join(data_path, 'train', r'news.tsv')
train_behaviors_file = os.path.join(data_path, 'train', r'behaviors.tsv')
valid_news_file = os.path.join(data_path, 'valid', r'news.tsv')
valid_behaviors_file = os.path.join(data_path, 'valid', r'behaviors.tsv')
wordEmb_file = os.path.join(data_path, "utils", "embedding.npy")
userDict_file = os.path.join(data_path, "utils", "uid2index.pkl")
wordDict_file = os.path.join(data_path, "utils", "word_dict.pkl")
yaml_file = os.path.join(data_path, "utils", r'lstur.yaml')

In [49]:
tmpdir = TemporaryDirectory()
data_path = r"C:\Users\super\OneDrive\Documents\SSIS\Swoopo"

train_news_file = os.path.join(data_path, 'train', r'news.tsv')
train_behaviors_file = os.path.join(data_path, 'train', r'behaviors.tsv')
valid_news_file = os.path.join(data_path, 'valid', r'news.tsv')
valid_behaviors_file = os.path.join(data_path, 'valid', r'behaviors.tsv')
wordEmb_file = os.path.join(data_path, "utils", "embedding.npy")
userDict_file = os.path.join(data_path, "utils", "uid2index.pkl")
wordDict_file = os.path.join(data_path, "utils", "word_dict.pkl")
yaml_file = os.path.join(data_path, "utils", r'lstur.yaml')

mind_url, mind_train_dataset, mind_dev_dataset, mind_utils = get_mind_data_set(MIND_type)

if not os.path.exists(train_news_file):
    download_deeprec_resources(mind_url, os.path.join(data_path, 'train'), mind_train_dataset)
    
if not os.path.exists(valid_news_file):
    download_deeprec_resources(mind_url, \
                               os.path.join(data_path, 'valid'), mind_dev_dataset)
if not os.path.exists(yaml_file):
    download_deeprec_resources(r'https://recodatasets.z20.web.core.windows.net/newsrec/', \
                               os.path.join(data_path, 'utils'), mind_utils)

100%|██████████| 17.0k/17.0k [00:01<00:00, 8.76kKB/s]
100%|██████████| 9.84k/9.84k [00:02<00:00, 4.46kKB/s]
100%|██████████| 95.0k/95.0k [00:20<00:00, 4.74kKB/s]


In [11]:
print(yaml_file)

C:\Users\super\OneDrive\Documents\SSIS\Swoopo\utils\lstur.yaml


In [25]:
hparams = prepare_hparams(yaml_file, 
                          wordEmb_file=wordEmb_file,
                          wordDict_file=wordDict_file, 
                          userDict_file=userDict_file,
                          batch_size=batch_size,
                          epochs=epochs)
print(hparams)

HParams object with values {'support_quick_scoring': True, 'dropout': 0.2, 'attention_hidden_dim': 200, 'head_num': 4, 'head_dim': 100, 'filter_num': 400, 'window_size': 3, 'vert_emb_dim': 100, 'subvert_emb_dim': 100, 'gru_unit': 400, 'type': 'ini', 'user_emb_dim': 50, 'learning_rate': 0.0001, 'optimizer': 'adam', 'epochs': 5, 'batch_size': 64, 'show_step': 100000, 'title_size': 30, 'his_size': 50, 'data_format': 'news', 'npratio': 4, 'metrics': ['group_auc', 'mean_mrr', 'ndcg@5;10'], 'word_emb_dim': 300, 'cnn_activation': 'relu', 'model_type': 'lstur', 'loss': 'cross_entropy_loss', 'wordEmb_file': 'C:\\Users\\super\\OneDrive\\Documents\\SSIS\\Swoopo\\utils\\embedding.npy', 'wordDict_file': 'C:\\Users\\super\\OneDrive\\Documents\\SSIS\\Swoopo\\utils\\word_dict.pkl', 'userDict_file': 'C:\\Users\\super\\OneDrive\\Documents\\SSIS\\Swoopo\\utils\\uid2index.pkl'}


In [26]:
iterator = MINDIterator

In [27]:
model = LSTURModel(hparams, iterator, seed=seed)

Tensor("conv1d_2/Relu:0", shape=(None, 30, 400), dtype=float32)
Tensor("att_layer2_2/Sum_1:0", shape=(None, 400), dtype=float32)


C:\Users\super\anaconda3\envs\python38\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [28]:
print(model.run_eval(valid_news_file, valid_behaviors_file))

0it [00:00, ?it/s]C:\Users\super\anaconda3\envs\python38\lib\site-packages\keras\engine\training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
2891it [02:34, 18.72it/s]
157it [06:19,  2.42s/it]
10000it [00:01, 6443.74it/s]


{'group_auc': 0.4994, 'mean_mrr': 0.174, 'ndcg@5': 0.1436, 'ndcg@10': 0.2188}


In [29]:
%%time
model.fit(train_news_file, train_behaviors_file, valid_news_file, valid_behaviors_file)

157it [2:35:02, 59.25s/it]  
2891it [02:48, 17.12it/s]
157it [06:13,  2.38s/it]
10000it [00:01, 5502.94it/s]


at epoch 1
train info: logloss loss:1.6172661462407203
eval info: group_auc:0.4981, mean_mrr:0.173, ndcg@10:0.2177, ndcg@5:0.1425
at epoch 1 , train time: 9302.1 eval time: 559.1


157it [33:16, 12.72s/it]
2891it [02:48, 17.19it/s]
157it [16:53:37, 387.37s/it] 
10000it [00:01, 5031.07it/s]


at epoch 2
train info: logloss loss:1.612929994892922
eval info: group_auc:0.4998, mean_mrr:0.176, ndcg@10:0.2208, ndcg@5:0.1456
at epoch 2 , train time: 1996.6 eval time: 61012.6


157it [4:27:12, 102.12s/it] 
2891it [02:51, 16.86it/s]
157it [06:42,  2.56s/it]
10000it [00:02, 4721.08it/s]


at epoch 3
train info: logloss loss:1.611287175470097
eval info: group_auc:0.5008, mean_mrr:0.1755, ndcg@10:0.2203, ndcg@5:0.1451
at epoch 3 , train time: 16032.3 eval time: 595.4


157it [7:36:11, 174.34s/it] 
2891it [03:08, 15.30it/s]
157it [06:55,  2.64s/it]
10000it [00:01, 5542.82it/s]


at epoch 4
train info: logloss loss:1.6114720387063968
eval info: group_auc:0.5008, mean_mrr:0.1728, ndcg@10:0.2174, ndcg@5:0.1422
at epoch 4 , train time: 27371.6 eval time: 626.9


157it [33:28, 12.79s/it]
2891it [02:52, 16.78it/s]
157it [06:17,  2.40s/it]
10000it [00:01, 6425.45it/s]


at epoch 5
train info: logloss loss:1.610877827474266
eval info: group_auc:0.5005, mean_mrr:0.1748, ndcg@10:0.2195, ndcg@5:0.1443
at epoch 5 , train time: 2008.2 eval time: 566.5
CPU times: total: 9h 10min 3s
Wall time: 1d 9h 21min 11s


In [30]:
%%time
res_syn = model.run_eval(valid_news_file, valid_behaviors_file)
print(res_syn)

2891it [03:04, 15.68it/s]
157it [06:20,  2.42s/it]
10000it [00:01, 6526.92it/s]


{'group_auc': 0.5005, 'mean_mrr': 0.1748, 'ndcg@5': 0.1443, 'ndcg@10': 0.2195}
CPU times: total: 20min 29s
Wall time: 9min 41s


In [31]:
sb.glue("res_syn", res_syn)

In [32]:
model_path = os.path.join(data_path, "model")
os.makedirs(model_path, exist_ok=True)

model.model.save_weights(os.path.join(model_path, "lstur_ckpt"))

In [33]:
model.model.save_weights("lstur_ckpt.h5")

In [ ]:
group_impr_indexes, group_labels, group_preds = model.run_fast_eval(valid_news_file, valid_behaviors_file)

2851it [03:13, 12.34it/s]

In [ ]:
with open(os.path.join(data_path, 'prediction.txt'), 'w') as f:
    for impr_index, preds in tqdm(zip(group_impr_indexes, group_preds)):
        impr_index += 1
        pred_rank = (np.argsort(np.argsort(preds)[::-1]) + 1).tolist()
        pred_rank = '[' + ','.join([str(i) for i in pred_rank]) + ']'
        f.write(' '.join([str(impr_index), pred_rank])+ '\n')

In [48]:
f = zipfile.ZipFile(os.path.join(data_path, 'prediction.zip'), 'w', zipfile.ZIP_DEFLATED)
f.write(os.path.join(data_path, 'prediction.txt'), arcname='prediction.txt')
f.close()

In [9]:
import pickle

In [17]:
with open(userDict_file, 'rb') as f:
    s = pickle.load(f)

In [18]:
s

{'+34661281XXX': 0,
 '+441479812XXX': 1,
 '+441698293XXX': 2,
 '+441757291XXX': 3,
 '+441786449XXX': 4,
 '+441924505XXX': 5,
 '+441924507XXX': 6,
 '+447787554XXX': 7,
 '+447824531XXX': 8,
 '+4915120602XXX': 9,
 '+491629827XXX': 10,
 '+491705480XXX': 11,
 '+491715776XXX': 12,
 '+491727914XXX': 13,
 '+4917663041XXX': 14,
 '+4917696393XXX': 15,
 '+491774590XXX': 16,
 '+491775452XXX': 17,
 '+491787255XXX': 18,
 '+492029469XXX': 19,
 '+492065907XXX': 20,
 '+4923819726XXX': 21,
 '+4925245XXX': 22,
 '+4925729233XXX': 23,
 '+492572943XXX': 24,
 '+492722639XXX': 25,
 '+493370129XXX': 26,
 '+494442959XXX': 27,
 '+49541388XXX': 28,
 '+4960217930XXX': 29,
 '+4968068670XXX': 30,
 '+4971619240XXX': 31,
 '+4977222XXX': 32,
 '+497731955XXX': 33,
 '+4977419657XXX': 34,
 '010363': 35,
 '019andre82': 36,
 '01grandprix': 37,
 '01greatbuy': 38,
 '0305545c': 39,
 '04spannerman': 40,
 '08023743': 41,
 '0815dragon': 42,
 '08luci08': 43,
 '09eragon': 44,
 '09riv': 45,
 '0ntari054': 46,
 '1103blacky': 47,
 '112

In [55]:
with open(userDict_file+"2", 'wb') as f:
    pickle.dump(s, f)

In [12]:
with open(wordDict_file, 'rb') as f:
    s = pickle.load(f)

In [13]:
s

{'the': 1,
 'brands': 2,
 'queen': 3,
 'elizabeth': 4,
 ',': 5,
 'prince': 6,
 'charles': 7,
 'and': 8,
 'philip': 9,
 'swear': 10,
 'by': 11,
 '50': 12,
 'worst': 13,
 'habits': 14,
 'for': 15,
 'belly': 16,
 'fat': 17,
 'cost': 18,
 'of': 19,
 'trump': 20,
 's': 21,
 'aid': 22,
 'freeze': 23,
 'in': 24,
 'trenches': 25,
 'ukraine': 26,
 'war': 27,
 'i': 28,
 'was': 29,
 'an': 30,
 'nba': 31,
 'wife': 32,
 '.': 33,
 'here': 34,
 'how': 35,
 'it': 36,
 'affected': 37,
 'my': 38,
 'mental': 39,
 'health': 40,
 'to': 41,
 'get': 42,
 'rid': 43,
 'skin': 44,
 'tags': 45,
 'according': 46,
 'a': 47,
 'dermatologist': 48,
 'should': 49,
 'nfl': 50,
 'be': 51,
 'able': 52,
 'fine': 53,
 'players': 54,
 'criticizing': 55,
 'officiating': 56,
 '?': 57,
 'been': 58,
 'orlando': 59,
 'hottest': 60,
 'october': 61,
 'ever': 62,
 'so': 63,
 'far': 64,
 'but': 65,
 'cooler': 66,
 'temperatures': 67,
 'on': 68,
 'way': 69,
 'chile': 70,
 'three': 71,
 'die': 72,
 'supermarket': 73,
 'fire': 74,
 'am